<a href="https://colab.research.google.com/github/suryasridhar/GoogleTrends-StockPrice/blob/develop/GoogleTrends_StockPrice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import yfinance as yf
from pytrends.request import TrendReq
import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time

def fetch_google_trends(ticker, timeframe='now 1-d'):
    pytrends = TrendReq(hl='en-US', tz=360)
    kw_list = [ticker]
    pytrends.build_payload(kw_list, cat=0, timeframe=timeframe, geo='', gprop='')

    # Add a delay to prevent hitting rate limits
    time.sleep(1)  # Wait 1 second between requests

    trends_data = pytrends.interest_over_time()
    if not trends_data.empty:
        return trends_data[ticker].iloc[-1]  # Return the most recent trend score
    else:
        return 0  # If no data available, return 0

# Fetch historical stock data using yfinance
def fetch_stock_data(ticker, days=30):
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days=days)
    stock = yf.Ticker(ticker)
    historical_data = stock.history(start=start_date, end=end_date)
    return historical_data

# Prepare training data: get historical stock data and Google Trends for each day
def prepare_training_data(ticker, days=30):
    historical_data = fetch_stock_data(ticker, days=days)
    dates = historical_data.index

    X = []
    y = []

    for date in dates[:-1]:
        google_trends_score = fetch_google_trends(ticker, timeframe=f'{date.strftime("%Y-%m-%d")} {date.strftime("%Y-%m-%d")}')

        high_today = historical_data.loc[date, 'High']
        low_today = historical_data.loc[date, 'Low']
        next_day = date + datetime.timedelta(days=1)

        if next_day in historical_data.index:
            max_next_day = historical_data.loc[next_day, 'High']
            X.append([high_today, low_today, google_trends_score])
            y.append(max_next_day)

    return np.array(X), np.array(y)

# Scale data using MinMaxScaler
def scale_data(X, y):
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_scaled = scaler_X.fit_transform(X)
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

    return X_scaled, y_scaled, scaler_X, scaler_y

# Define and train a neural network model
def train_neural_network(X, y, epochs=50, batch_size=8):
    model = Sequential([
        Dense(64, input_dim=X.shape[1], activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='linear')
    ])

    model.compile(optimizer='adam', loss='mse')
    model.fit(X, y, epochs=epochs, batch_size=batch_size, verbose=1)

    return model

# Predict using the trained neural network
def predict_next_day_max_nn(model, scaler_X, scaler_y, high_today, low_today, google_trends_score):
    X_input = np.array([[high_today, low_today, google_trends_score]])
    X_scaled = scaler_X.transform(X_input)
    y_pred_scaled = model.predict(X_scaled)
    return scaler_y.inverse_transform(y_pred_scaled)[0, 0]

# Backtest the neural network model
def backtest_neural_network(ticker, model, scaler_X, scaler_y, days=10):
    historical_data = fetch_stock_data(ticker, days=days + 1)
    results = []

    for i in range(days):
        date_today = historical_data.index[-(days - i)]
        date_tomorrow = historical_data.index[-(days - i - 1)]

        high_today = historical_data.loc[date_today, 'High']
        low_today = historical_data.loc[date_today, 'Low']
        actual_max = historical_data.loc[date_tomorrow, 'High']

        # Fetch Google Trends score for the current day
        google_trends_score = fetch_google_trends(ticker, timeframe=f'{date_today.strftime("%Y-%m-%d")} {date_today.strftime("%Y-%m-%d")}')

        # Predict the next day's max price
        predicted_max = predict_next_day_max_nn(model, scaler_X, scaler_y, high_today, low_today, google_trends_score)

        results.append({
            "Date": date_tomorrow.strftime('%Y-%m-%d'),
            "Predicted Max": predicted_max,
            "Actual Max": actual_max
        })

    return pd.DataFrame(results)

# Main function to run the script
def main(ticker_symbol):
    print(f"Fetching data and Google Trends analysis for {ticker_symbol}...")

    # Prepare training data
    X, y = prepare_training_data(ticker_symbol, days=30)
    X_scaled, y_scaled, scaler_X, scaler_y = scale_data(X, y)

    # Train neural network
    print("Training neural network...")
    model = train_neural_network(X_scaled, y_scaled)

    # Predict max price for today
    today_data = fetch_stock_data(ticker_symbol, days=1)
    high_today = today_data['High'].values[-1]
    low_today = today_data['Low'].values[-1]
    google_trends_score = fetch_google_trends(ticker_symbol, timeframe='now 1-d')
    predicted_max_price = predict_next_day_max_nn(model, scaler_X, scaler_y, high_today, low_today, google_trends_score)

    print(f"**Predicted Max Price for Today ({ticker_symbol}):** ${predicted_max_price:.2f}")

    # Backtest for the last 10 days
    print(f"Backtesting for {ticker_symbol} over the last 10 days...")
    backtest_results = backtest_neural_network(ticker_symbol, model, scaler_X, scaler_y, days=10)

    print(backtest_results)
    return backtest_results

# Example usage
if __name__ == "__main__":
    ticker = input("Enter the stock ticker symbol (e.g., AAPL, MSFT): ")
    main(ticker)


Enter the stock ticker symbol (e.g., AAPL, MSFT): MSFT
Fetching data and Google Trends analysis for MSFT...


TooManyRequestsError: The request failed: Google returned a response with code 429